In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 100,
                                                 class_mode = 'binary')

Found 7999 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 100,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [4]:
cnn = tf.keras.models.Sequential()

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
cnn.add(tf.keras.layers.Flatten())

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [10]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [11]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


80/80 [==============================] - 47s 553ms/step - loss: 0.7138 - accuracy: 0.5208 - val_loss: 0.6830 - val_accuracy: 0.5520
Epoch 2/25
80/80 [==============================] - 40s 497ms/step - loss: 0.6655 - accuracy: 0.5952 - val_loss: 0.6317 - val_accuracy: 0.6345
Epoch 3/25
80/80 [==============================] - 40s 506ms/step - loss: 0.6265 - accuracy: 0.6512 - val_loss: 0.6048 - val_accuracy: 0.6675
Epoch 4/25
80/80 [==============================] - 37s 464ms/step - loss: 0.5932 - accuracy: 0.6811 - val_loss: 0.5563 - val_accuracy: 0.7195
Epoch 5/25
80/80 [==============================] - 38s 480ms/step - loss: 0.5576 - accuracy: 0.7138 - val_loss: 0.5942 - val_accuracy: 0.6895
Epoch 6/25
80/80 [==============================] - 37s 467ms/step - loss: 0.5532 - accuracy: 0.7145 - val_loss: 0.5385 - val_accuracy: 0.7330
Epoch 7/25
80/80 [==============================] - 39s 486ms/step - loss: 0.5357 - accuracy: 0.7267 - val_loss: 0.5080 - val_accuracy: 0.76

In [ ]:
import cv2
import numpy as np
from keras.preprocessing import image

cap = cv2.VideoCapture(0)
cap.set(3, 640)  # set Width
cap.set(4, 480)  # set Height

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Process the frame for prediction
    img = cv2.resize(frame, (64, 64))  # Resize image to match the model input size
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = img / 255.0  # Normalize pixel values
    
    # Make prediction
    result = cnn.predict(img)
    if result[0][0] < 0.5:
        prediction = 'Cat'
    else:
        prediction = 'Dog'
    
    # Display prediction
    cv2.putText(frame, "Predicted class: " + prediction, (20, 20), cv2.FONT_HERSHEY_COMPLEX, 1, (100, 200, 56), 2)
    cv2.imshow("Video Output", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
print(prediction)